In [ ]:
!pip install networkx powerlaw

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import statistics
import sys
import random
import math
import numpy as np
import powerlaw
import pandas as pd
import seaborn as sns
from itertools import chain, combinations
from scipy.cluster.hierarchy import dendrogram
from collections import Counter
from scipy.stats import norm
from termcolor import colored, cprint
from matplotlib.patches import Rectangle

In [ ]:
MORENO_RESIDENCE_HALL_FILE = 'networks/moreno_residence_hall.txt'

# Residence-Hall-Network

(Directed)

His directed network contains friendship ratings between 217 residents living at a residence hall located on the Australian National University campus. A node represents a person and 
an edge represents a friendship tie.  The friendships are weighted as follows from strongest to weakest tie:  5 (best friend), 4 (close friend), 3 (friend), 2, 1.

In [ ]:
G = nx.read_weighted_edgelist(MORENO_RESIDENCE_HALL_FILE, create_using= nx.DiGraph, nodetype=int)
cprint(nx.info(G),'green')

In [ ]:
def draw_graph(pos,nodecolors):
    plt.figure(3, figsize=(100, 100))
    edges, weights = zip(*nx.get_edge_attributes(G, 'weight').items())
    nx.draw(G,node_color=nodecolors, node_size=8000,arrowsize=40, pos=pos, edgelist = edges, edge_color = weights, edge_cmap = plt.cm.Greys, edge_vmin=0.0, edge_vmax=5.0, width=2.0)
    nx.draw_networkx_labels(G, font_size=30, pos=pos)
    ax=plt.gca()
    ax.collections[0].set_edgecolor("#000000")
    plt.show()


In [ ]:
pos = nx.spring_layout(G, k=0.6, iterations=100)
draw_graph(pos,'#add8e6')

In [ ]:
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
nodes = G.nodes()
edges = G.edges()

## Random models

### Erdös-Rényi random graph model

In [ ]:
n = G.number_of_nodes()
m = G.number_of_edges()
seed = 20160  # seed random number generators for reproducibility

# Use seed for reproducibility
ErdG= nx.gnm_random_graph(n, m, seed=seed, directed=True)

In [ ]:
pos = nx.spring_layout(ErdG, seed=seed, k=0.6, iterations=100)  # Seed for reproducible layout
plt.figure(3, figsize=(100, 100))
nx.draw(ErdG, node_color='#add8e6', node_size=8000,arrowsize=40, pos=pos, width=2.0)
nx.draw_networkx_labels(ErdG, font_size=30, pos=pos)
ax=plt.gca()
ax.collections[0].set_edgecolor("#000000")
plt.show()

In [ ]:
# some properties
""" print("node degree clustering")
for v in nx.nodes(ErdG):
    print(f"{v} {nx.degree(ErdG, v)} {nx.clustering(ErdG, v)}")

print()
print("the adjacency list")
for line in nx.generate_adjlist(ErdG):
    print(line)

"""
#average shoart path length (for small world property)
avg_short_path = nx.average_shortest_path_length(ErdG)
cprint("Average shortest path length Erdos Random network: ", 'blue', end=' ')
cprint(round(avg_short_path,2), 'green')

# (1) Distances between pairs of nodes are short (small-world property): good!

if(math.isclose(math.log(ErdG.number_of_nodes(),10),avg_short_path,abs_tol=0.5)):
    cprint("Small-word", 'yellow')
else:
    cprint("NOT Small-word ", 'yellow')

# (2) The average clustering coefficient is much lower than on real networks of the  same size and average degree: bad!
avg_clustering = nx.average_clustering(G)
avg_clustering_ErdG = nx.average_clustering(ErdG)
print("Average Clustering Erdos Random network: ", avg_clustering_ErdG)
print("Average Clustering our Network: ", avg_clustering)

# (3) The nodes have approximately the same degree, there are no hubs: bad! 




### The Watts-Strogatz model (Small World)

In [ ]:
degrees_sequences = [G.degree(n) for n in G.nodes]
median_degree = statistics.median(degrees_sequences)
Watts_Strogatz_G = nx.watts_strogatz_graph(G.number_of_nodes(), int(median_degree), 0.1) 
plt.figure(figsize = (100, 100))
nx.draw_circular(Watts_Strogatz_G, with_labels=True)

In [ ]:
# (1) There is a range of values of the rewiring probability p for which distances  between pairs of nodes are short (small-world property)
#  and the average clustering coefficient is high: good!



# (2) The nodes have approximately the same degree, there are no hubs: bad!